# <B> Clean up </B>

## AutoReload

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## clean

In [10]:
import os
import time
import boto3
from utils.ssm import parameter_store

In [11]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)
strPrefix = pm.get_params(key="PREFIX")

* S3 Bucket

In [12]:
strBucketName = pm.get_params(key="-".join([strPrefix, "BUCKET"]))

In [13]:
strQuery = "".join(["aws s3 rm --recursive ", '"s3://', strBucketName, '"'])
strResponse = os.popen(strQuery).read()
strQuery = "".join(["aws s3 rb ", '"s3://', strBucketName, '"'])
strResponse = os.popen(strQuery).read()

remove_bucket failed: s3://sagemaker-us-east-1-419974056037 An error occurred (AccessDenied) when calling the DeleteBucket operation: Access Denied


* pipeline

In [14]:
strPipelineName = pm.get_params(key="-".join([strPrefix, "PIPELINE-NAME"]))
strPipelineName

IndexError: list index out of range

In [24]:
!aws sagemaker delete-pipeline --pipeline-name $strPipelineName 

{
    "PipelineArn": "arn:aws:sagemaker:us-east-1:419974056037:pipeline/pipeline-train-model-b"
}


* model registry

In [25]:
sagemaker_client = boto3.client('sagemaker')

In [26]:
def empty_and_delete_model_package(sagemaker_client, mpg_name):
    mpg = sagemaker_client.list_model_packages(
        ModelPackageGroupName=mpg_name,
    )
    
    # Delete model packages if Group not empty
    model_packages = mpg.get('ModelPackageSummaryList')
    if model_packages:
        for mp in model_packages:
            sagemaker_client.delete_model_package(
                ModelPackageName=mp['ModelPackageArn']
            )
            time.sleep(1)

    # Delete model package group
    sagemaker_client.delete_model_package_group(
        ModelPackageGroupName=mpg_name
    )

In [28]:
DeleteModelGroupName = pm.get_params(key="-".join([strPrefix, "MODEL-GROUP-NAME"]))
empty_and_delete_model_package(sagemaker_client, DeleteModelGroupName)

* code commit, buldand pipeline

In [32]:
from utils.code_pipeline import code_pipeline_handler
cph = code_pipeline_handler()

In [33]:
cph.delete_repository(strRepoName=pm.get_params(key="-".join([strPrefix, "MLOPS-COMMIT-NAME"])))
cph.delete_build_project(strCodeBuildPJTName=pm.get_params(key="-".join([strPrefix, "MLOPS-BUILD-NAME"])))
cph.delete_pipeline(strPipeLineName=pm.get_params(key="-".join([strPrefix, "MLOPS-PIPELINE-NAME"])))

== DELETE REPO ==
  Repository name [REPO-DJ-SM-PIPELINE] was successfully deleted!!
''
''


* event rule

In [61]:
client_events = boto3.client('events')

In [62]:
response = client_events.remove_targets(
    Rule=pm.get_params(key="-".join([strPrefix, "EVENT-RULE-NAME"])),
    Ids=[pm.get_params(key="-".join([strPrefix, "LAMBDA-PIPELINE-MONITOR"]))],
    Force=True
)
response

{'FailedEntryCount': 0,
 'FailedEntries': [],
 'ResponseMetadata': {'RequestId': 'fa76e790-5099-427a-ba42-e7dd48f41265',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fa76e790-5099-427a-ba42-e7dd48f41265',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '41',
   'date': 'Mon, 15 May 2023 07:07:36 GMT'},
  'RetryAttempts': 0}}

In [63]:
response = client_events.delete_rule(
    Name=pm.get_params(key="-".join([strPrefix, "EVENT-RULE-NAME"])),
    Force=True
)
response

{'ResponseMetadata': {'RequestId': '499ecf23-62aa-4172-84c3-2e353850f094',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '499ecf23-62aa-4172-84c3-2e353850f094',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Mon, 15 May 2023 07:07:36 GMT'},
  'RetryAttempts': 0}}

* Lambda

In [64]:
from utils.lambda_func import lambda_handler

In [65]:
lam_handler = lambda_handler(region_name=strRegionName)

In [66]:
lam_handler.delete_function(pm.get_params(key="-".join([strPrefix, "LAMBDA-PIPELINE-MONITOR"])))

  lambda function: [DJ-SM-PIPELINE-LAMBDA] is deleted successfully


* SNS
    - using console

* parameter store

In [67]:
listParams = pm.get_all_params()
listParams = [strParam for strParam in listParams if strPrefix in strParam] + ["PREFIX"]

total_size = len(listParams)
for idx in range(0, total_size, 10):
    start = idx
    if start + 10 < total_size: end = start + 10
    else: end = total_size
    listSubParams = listParams[start:end]
    pm.delete_param(listSubParams)

  parameters: ['DJ-SM-PIPELINE-ACCOUNT-ID', 'DJ-SM-PIPELINE-BUCKET', 'DJ-SM-PIPELINE-CODE-BUILD-ROLE-ARN', 'DJ-SM-PIPELINE-CODE-EVENTBRIDGE-ROLE-ARN', 'DJ-SM-PIPELINE-CODE-PIPELINE-ROLE-ARN', 'DJ-SM-PIPELINE-CODE-REPO', 'DJ-SM-PIPELINE-CODECOMMIT-PWD', 'DJ-SM-PIPELINE-CODECOMMIT-USERNAME', 'DJ-SM-PIPELINE-DATA-PATH', 'DJ-SM-PIPELINE-EVENT-RULE-NAME'] is deleted successfully
  parameters: ['DJ-SM-PIPELINE-IMAGE-URI', 'DJ-SM-PIPELINE-IMAGE-URI-PREP', 'DJ-SM-PIPELINE-IMAGE-URI-TR', 'DJ-SM-PIPELINE-LAMBDA-PIPELINE-MONITOR', 'DJ-SM-PIPELINE-LAMBDA-ROLE-ARN', 'DJ-SM-PIPELINE-MLOPS-BUILD-NAME', 'DJ-SM-PIPELINE-MLOPS-COMMIT-NAME', 'DJ-SM-PIPELINE-MLOPS-PIPELINE-NAME', 'DJ-SM-PIPELINE-MODEL-GROUP-NAME', 'DJ-SM-PIPELINE-PIPELINE-ARN'] is deleted successfully
  parameters: ['DJ-SM-PIPELINE-REGION', 'DJ-SM-PIPELINE-SAGEMAKER-ROLE-ARN', 'DJ-SM-PIPELINEPIPELINE-NAME', 'PREFIX'] is deleted successfully
